In [2]:
import pandas as pd
import numpy as np
from datasets import Dataset

pd.options.mode.chained_assignment = None


def create_dataset(Scenario = {'train':[], 'test':[], 'scenario':1}, decop=None, mem = None, intent = None, seed=42, cv=10):

  train={f'split_{i+1}':[] for i in range(cv)}
  test={f'split_{i+1}':[] for i in range(cv)}

  np.random.seed(seed)

  de_arr = np.arange(1, 501)
  np.random.shuffle(de_arr)
  de_arr_r = de_arr.reshape((cv, int(500/cv)))

  num_row = (mem.index.shape[0] - (mem.shape[0]%cv))
  me_arr = np.random.choice(mem.index.shape[0], int(num_row), replace=False)
  me_arr_r = me_arr.reshape((cv, int(num_row/cv)))

  in_arr = np.arange(0, intent.index.shape[0])
  np.random.shuffle(in_arr)
  in_arr_r = in_arr.reshape((cv, int(intent.shape[0]/cv))) ### fin qui testata

  cv_ind = 0
  for k in train.keys():

    if type(decop) != None:

      if Scenario['scenario'] == 1 or Scenario['scenario'] == 3:

        ind = de_arr_r[cv_ind,:]
        antind = [i for i in de_arr if i not in ind]

        decop_train = decop[decop['ID'].isin(antind)]
        decop_test = decop[decop['ID'].isin(ind)]

        train[k].append(decop_train)
        test[k].append(decop_test)


      else:

        if 'decop' in Scenario['train']:
          train[k].append(decop)
        else:
          test[k].append(decop)

    if type(mem) !=None:

      if Scenario['scenario'] == 1 or Scenario['scenario'] == 3:

        ind = me_arr_r[cv_ind,:]
        antind = [i for i in me_arr if i not in ind]

        mem_train = mem[mem.index.isin(antind)]
        mem_test = mem[mem.index.isin(ind)]

        train[k].append(mem_train)
        test[k].append(mem_test)

      else:

        if 'hyppocorpus' in Scenario['train']:
          train[k].append(mem)
        else:
          test[k].append(mem)



    if type(intent) != None:

      if Scenario['scenario'] == 1 or Scenario['scenario'] == 3:

        ind = in_arr_r[cv_ind,:]
        antind = [i for i in in_arr if i not in ind]

        intent_train = intent[intent.index.isin(antind)]
        intent_test = intent[intent.index.isin(ind)]

        train[k].append(intent_train)
        test[k].append(intent_test)


      else:

        if 'intention' in Scenario['train']:
          train[k].append(intent)
        else:
          test[k].append(intent)


    if Scenario['scenario'] == 1 or Scenario['scenario'] == 3:
      train[k] = pd.concat(train[k])
      test[k] = pd.concat(test[k])
    else:
      train[k] = pd.concat(train[k])
      test[k] = test[k][0]

    cv_ind+=1



  return train, test

def get_s3_data(drive_path):

#########################################################################################################
  dc = pd.read_excel(f"DecOp_data_EN_500.xlsx")

  d1 = []
  for i in range(dc.shape[0]):
    row = dc.iloc[i]
    d1.append({'ID': row['ID'],
        'age' :row['age'], 'gender': row['gender'],
    'sent' : row['A'].replace('\n', " ") ,
    'labels'  : row['GT.A']})

    d1.append({'ID': row['ID'],
        'age' :row['age'], 'gender': row['gender'],
    'sent' : row['E'].replace('\n', " ")  ,
    'labels'  : row['GT.E']})
    d1.append({'ID': row['ID'],
        'age' :row['age'], 'gender': row['gender'],
    'sent' : row['GM'].replace('\n', " ")  ,
    'labels'  : row['GT.GM']})

    d1.append({'ID': row['ID'],
        'age' :row['age'], 'gender': row['gender'],
    'sent' : row['Pom'].replace('\n', " ")  ,
    'labels'  : row['GT.Pom']})

    d1.append({'ID': row['ID'],
        'age' :row['age'], 'gender': row['gender'],
    'sent' : row['CL'].replace('\n', " ")  ,
    'labels'  : row['GT.CL']})

    decop = pd.DataFrame.from_records(d1)

    decop=decop[['ID', 'sent', 'labels']]
    decop['type']='A'
#########################################################################################################
  hc = memories_df = pd.read_excel(f"hcV3-stories.xlsx")

  mem = hc[hc['memType']!='retold']
  mem = mem.dropna(subset=['story', 'memType'])
  mem = mem[['story', 'memType']]
  mem['memType'][mem['memType']=='recalled']='T'
  mem['memType'][mem['memType']=='imagined']='F'
  mem['story_length']=mem['story'].map(len)
  mean=mem['story_length'].describe()['mean']
  std=mem['story_length'].describe()['std']
  min=mean-2.5*std
  mem=mem[ min < mem['story_length'] ][['story', 'memType']]
  mem = mem.rename(columns={'story': 'sent', 'memType': 'labels'})
  mem['type']='B'
#########################################################################################################
  intent = pd.read_csv(f'sign_events_data_statements.csv', encoding = "ISO-8859-1")

  intent['outcome_class'][intent['outcome_class']=='t']='T'
  intent['outcome_class'][intent['outcome_class']=='d']='F'
  intent['q1']=intent['q1'].apply(lambda x: x.replace('\n', ''))
  intent = intent.rename(columns={'q1': 'sent', 'outcome_class': 'labels'})
  intent = intent[['sent', 'labels']]
  intent['type']='C'
#########################################################################################################

  seed=42
  np.random.seed(seed)
  numcv=10

  dataset = ['decop', 'hyppocorpus', 'intention']

  Scenario = {'train': dataset, 'test': dataset,  'scenario': 3}


  train, test = create_dataset(Scenario, decop=decop, mem=mem, intent=intent, seed=seed, cv=numcv)

  return train, test

In [ ]:
!pip install datasets
!pip install accelerate -U
!pip install evaluate
!pip install transformers[torch]

In [ ]:
! cp /content/drive/MyDrive/ColabNotebooks/get_data.py /content

In [3]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, logging
from datasets import Dataset, load_metric
import evaluate
#from get_data import get_s3_data

logging.set_verbosity_warning()

os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

drive_path = "/content/drive/MyDrive/ColabNotebooks"

train, test = get_s3_data(drive_path)

In [4]:
for sp in train.keys():

    train[sp]['labels'] = train[sp]['labels'].map({'F': 0, 'T': 1})

In [5]:
for sp in test.keys():

    test[sp]['labels'] = test[sp]['labels'].map({'F': 0, 'T': 1})

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["sent"], padding="max_length", truncation=True)

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
metric = evaluate.load("accuracy")

In [11]:
num_epochs = 2

collect_result={}
results={}
types={}
preds={}
trus={}


for sp in tqdm(train.keys()):


  data_train = Dataset.from_pandas(train[sp])
  data_test = Dataset.from_pandas(test[sp])

  checkpoint = f"bert-base-cased"

  tokenizer = AutoTokenizer.from_pretrained(f"{checkpoint}")

  model = AutoModelForSequenceClassification.from_pretrained(f"{checkpoint}", num_labels=2)

  data_train = data_train.map(tokenize_function, batched=True)
  data_test = data_test.map(tokenize_function, batched=True)

  #training_args = TrainingArguments(output_dir="test_trainer", num_train_epochs=num_epochs, evaluation_strategy="epoch")

  training_args = TrainingArguments(
        output_dir="test_trainer",
        num_train_epochs=num_epochs,
        evaluation_strategy="epoch",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16)

  trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_train,
    eval_dataset=data_test,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

  trainer.train()


  predss=trainer.predict(data_test)

  metric = load_metric("glue", "mrpc")
  predictions = np.argmax(predss.predictions, axis=-1)

  #print(metric.compute(predictions=predictions, references=predss.label_ids))


  pred = predictions

  types[sp] = data_test['type']
  preds[sp] = pred
  trus[sp] = np.array(test[sp]['labels'])

  collect_result[sp]=pd.DataFrame.from_dict({"prediction": pred, 'labels': np.array(test[sp]['labels']), 'type': test[sp]['type']})
  collect_result[sp]['corr'] = collect_result[sp]['prediction'] == collect_result[sp]['labels']
  results[sp] = [(pred == trus[sp]).mean()]
  del model
  del trainer



results = pd.DataFrame.from_dict(results)

print('Average accuracy: ', results.mean(axis=1))
print('Standard deviation of accuracy: ', results.std(axis=1))
print('Max accuracy: ', results.max(axis=1))
print('Min deviation of accuracy: ', results.min(axis=1))
show_result = pd.concat(collect_result.values(), keys=collect_result.keys())
show_result = show_result.droplevel(1)

print("Average accuracy typewise")
display(show_result.groupby([show_result.index, 'type']).mean().groupby('type')['corr'].mean())
print("Standard deviation of accuracy typewise")
display(show_result.groupby([show_result.index, 'type']).mean().groupby('type')['corr'].std())
print("Max accuracy typewise")
display(show_result.groupby([show_result.index, 'type']).mean().groupby('type')['corr'].max())
print("Min accuracy typewise")
display(show_result.groupby([show_result.index, 'type']).mean().groupby('type')['corr'].min())

  0%|          | 0/10 [00:00<?, ?it/s]Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/7690 [00:00<?, ? examples/s]

Map:   0%|          | 0/847 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.608929,0.676505,0.727092
2,0.609900,0.517374,0.761511,0.753056


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
 10%|█         | 1/10 [08:19<1:14:55, 499.47s/it]Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

type
A    0.026587
B    0.013357
C    0.035321
Name: corr, dtype: float64

Max accuracy typewise


type
A    0.856000
B    0.787472
C    0.780488
Name: corr, dtype: float64

Min accuracy typewise


type
A    0.764000
B    0.748299
C    0.658537
Name: corr, dtype: float64